<a href="https://colab.research.google.com/github/sunghwan-moon/2021_capstone_BrainAgePrediction_by_Federated_Learning/blob/main/FL_Simple_Data_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#!pip install tensorflow_federated
#!pip install nilearn
#!pip install transformations
#!pip install nest_asyncio

In [7]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import threading
from random import gauss
from scipy.ndimage.interpolation import map_coordinates
import operator
from scipy.ndimage.interpolation import shift,rotate
import pandas as pd
import os
import scipy.ndimage as nd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt
import math
import sys
import random
import glob
import nibabel as nib
from collections import defaultdict
from tensorflow.keras.utils import Sequence
import nest_asyncio
nest_asyncio.apply()
#from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, add, concatenate,GlobalAveragePooling3D,Conv3D,LeakyReLU,ELU, MaxPooling3D,AveragePooling3D
from tensorflow.keras.regularizers import l2 as L2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD,Adagrad
from keras.engine.topology import Layer
from keras.layers import BatchNormalization
from keras.layers import Input, InputLayer, InputSpec
#from tensorflow.keras import backend as K
#from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
#import tensorflow.compat.v1 as tf
# tf.compat.v1.enable_v2_behavior()
# tf.compat.v1.enable_eager_execution()

from tensorflow.compat.v1 import keras
from tensorflow.compat.v1.keras.layers import Dense, Dropout, Activation, Flatten, Input, add, concatenate,GlobalAveragePooling3D,Conv3D,LeakyReLU,ELU, MaxPooling3D,AveragePooling3D
from tensorflow.keras import layers

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
def build_local_model():
  model = keras.Sequential([
   layers.Dense(1, input_shape=(69,), activation = 'linear'),
   layers.Dense(10),
   layers.Dense(5),
   layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.Adam(0.001)

  model.compile(loss='mae',
               optimizer=optimizer,
               metrics=['mae'])

  return model

In [10]:
def getData(path):
  original_data = pd.read_excel(path)
  original_data.dropna(inplace=True)
  original_data.drop(["ID","Sex"], axis = 1, inplace=True)

  train, test = train_test_split(original_data, test_size = 0.2, random_state =1000) #100, 1000


  x_train = train.drop(["Age"],axis=1).to_numpy(np.float32)
  y_train = train["Age"].to_numpy(np.float32)

  x_test = test.drop(["Age"],axis=1).to_numpy(np.float32)
  y_test = test["Age"].to_numpy(np.float32)

  return x_train, y_train, x_test, y_test

In [11]:
x_train, y_train, x_test, y_test = getData('/content/drive/MyDrive/소융_캡스톤/뇌 MRI 사진_교수님/IXI_train_test_age.xlsx')

In [12]:
num_train_image = len(x_train)
num_test_image= len(x_test)
print("Train set: ", num_train_image)
print("Test set: ", num_test_image)
print("Train set Shape: ", x_train.shape)

Train set:  370
Test set:  93
Train set Shape:  (370, 69)


In [13]:
#client Data Non-iid 형태로 만들기 // 중복X

def Make_data_to_Non_RE_Client_data(num_train_image, Client_num, x_train, y_train):
  client_train_dataset = collections.OrderedDict()
  partition_Start = 0
  partition_End = int(num_train_image/Client_num - 1)

  for i in range(0, Client_num):
    client_key = "client_" + str(i+1) # client_1, client_2 .....

    train_data = collections.OrderedDict((('features', x_train[partition_Start:partition_End]),('label', y_train[partition_Start:partition_End])))
    client_train_dataset[client_key] = train_data
    partition_Start = partition_End + 1
    partition_End = int(partition_End + num_train_image/Client_num - 1)

  #tensorSliceClientData를 simulation의 instance로 변환
  train_dataset = tff.simulation.datasets.TestClientData(client_train_dataset)

  return train_dataset

In [14]:
#client Data Non-iid 형태로 만들기 // 중복 허용

def Make_data_to_RE_Client_data(num_train_image, Client_num, x_train, y_train):
  client_train_dataset = collections.OrderedDict()
  for i in range(0, Client_num):
    client_key = "client_" + str(i+1) # client_1, client_2 .....

  # non-iid를 위한 random sampling
    front_train = random.randint(1, num_train_image)
    rear_train = random.randint(front_train, num_train_image)

    train_data = collections.OrderedDict((('features', x_train[front_train:rear_train]),('label', y_train[front_train:rear_train])))
    client_train_dataset[client_key] = train_data

  #tensorSliceClientData를 simulation의 instance로 변환
  train_dataset = tff.simulation.datasets.TestClientData(client_train_dataset)

  return train_dataset

In [15]:
#총 Client 수 결정
Client_num = 4
Original_Client_data = Make_data_to_Non_RE_Client_data(num_train_image, Client_num, x_train, y_train)
Test_Client_data = Make_data_to_Non_RE_Client_data(num_test_image, 1, x_train, y_train)

In [16]:
# 각 Client 별로 학습할 데이터 추출

NUM_EPOCHS = 10
BATCH_SIZE = 100 #각 Client가 가질 데이터 수
SHUFFLE_BUFFER = 10
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(element):
    return (tf.reshape(element["features"], [-1,69]),
            tf.reshape(element["label"], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).prefetch(PREFETCH_BUFFER).map(batch_format_fn)

In [17]:
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]

In [18]:
sample_clients = Original_Client_data.client_ids[0:Client_num]
test_Clients = Test_Client_data.client_ids[0:1]

federated_train_data = make_federated_data(Original_Client_data, sample_clients)
federated_test_data = make_federated_data(Test_Client_data, test_Clients)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('Number of test datasets: {l}'.format(l=len(federated_test_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))
print("input_spec : {n}".format(n = federated_train_data[0].element_spec))

Number of client datasets: 4
Number of test datasets: 1
First dataset: <MapDataset shapes: ((None, 69), (None, 1)), types: (tf.float32, tf.float32)>
input_spec : (TensorSpec(shape=(None, 69), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))


In [19]:
def build_model():
  model = keras.Sequential([
   layers.Dense(1, input_shape=(69,), activation = 'linear'),
   layers.Dense(10),
   layers.Dense(5),
   layers.Dense(1)
  ])
  return model

In [20]:
def model_fn():
    keras_model = build_model()
    return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.MeanAbsoluteError(),
      metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [21]:
type(model_fn())

tensorflow_federated.python.learning.model_utils.EnhancedModel

In [22]:
def initialize_fn():
  model = model_fn()
  return model.trainable_variables

In [23]:
def next_fn(server_weights, federated_dataset):
  server_weights_at_client = broadcast(server_weights)
  client_weights = client_update(federated_dataset, server_weights_at_client)
  mean_client_weights = mean(client_weights)
  server_weights = server_update(mean_client_weights)
  
  return server_weights

In [24]:
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  client_weights = model.trainable_variables
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)
  
  for batch in dataset:
      with tf.GradientTape() as tape:
        outputs = model.forward_pass(batch)

      grads = tape.gradient(outputs.loss, client_weights)
      grads_and_vars = zip(grads, client_weights)

      client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

In [25]:
@tf.function
def server_update(model, mean_client_weights):
  model_weights = model.trainable_variables
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

In [26]:
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables

In [27]:
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(),tff.SERVER)

In [28]:
tf_dataset_type = tff.SequenceType(model_fn().input_spec)

In [29]:
str(tf_dataset_type)

'<float32[?,69],float32[?,1]>*'

In [30]:
model_weights_type = server_init.type_signature.result

In [31]:
str(model_weights_type)

'<float32[69,1],float32[1],float32[1,10],float32[10],float32[10,5],float32[5],float32[5,1],float32[1]>'

In [32]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) #Adam, SGD
  return client_update(model, tf_dataset, server_weights, client_optimizer)

In [33]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)

In [34]:
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

In [35]:
str(federated_dataset_type)

'{<float32[?,69],float32[?,1]>*}@CLIENTS'

In [36]:
str(federated_server_type)

'<float32[69,1],float32[1],float32[1,10],float32[10],float32[10,5],float32[5],float32[5,1],float32[1]>@SERVER'

In [37]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  server_weights_at_client = tff.federated_broadcast(server_weights)
  client_weights = tff.federated_map(
        client_update_fn, (federated_dataset, server_weights_at_client))
  mean_client_weights = tff.federated_mean(client_weights)
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights

In [38]:
federated_algorithm = tff.templates.IterativeProcess(
      initialize_fn=initialize_fn,
      next_fn=next_fn
    )

In [39]:
str(federated_algorithm.initialize.type_signature)

'( -> <float32[69,1],float32[1],float32[1,10],float32[10],float32[10,5],float32[5],float32[5,1],float32[1]>@SERVER)'

In [40]:
str(federated_algorithm.next.type_signature)

'(<server_weights=<float32[69,1],float32[1],float32[1,10],float32[10],float32[10,5],float32[5],float32[5,1],float32[1]>@SERVER,federated_dataset={<float32[?,69],float32[?,1]>*}@CLIENTS> -> <float32[69,1],float32[1],float32[1,10],float32[10],float32[10,5],float32[5],float32[5,1],float32[1]>@SERVER)'

In [41]:
def test_evaluate(server_state,Client_num):
  keras_model = build_model()
  keras_model.compile(
      loss=tf.keras.losses.MeanAbsoluteError(),
      metrics=[tf.keras.metrics.MeanAbsoluteError()]
  )
  keras_model.set_weights(server_state)
  for i in range(Client_num):
    print("Client{num}: ".format(num = i+1),"")
    hist = keras_model.evaluate(federated_train_data[i])
    print("")

In [42]:
def train_evaluate(server_state,Client_num):
  keras_model = build_model()
  keras_model.compile(
      loss=tf.keras.losses.MeanAbsoluteError(),
      metrics=[tf.keras.metrics.MeanAbsoluteError()]
  )
  keras_model.set_weights(server_state)
  for i in range(Client_num):
    print("Client{num}: ".format(num = i+1),"")
    hist = keras_model.evaluate(federated_train_data[i])
    print("")

In [43]:
server_state = federated_algorithm.initialize()

In [44]:
train_evaluate(server_state,Client_num)

Client1:  
10/10 [==============================] - 0s 2ms/step - loss: 173.8598 - mean_absolute_error: 173.8598

Client2:  
9/9 [==============================] - 0s 2ms/step - loss: 175.0705 - mean_absolute_error: 175.0705

Client3:  
9/9 [==============================] - 0s 2ms/step - loss: 172.2465 - mean_absolute_error: 172.2465

Client4:  
9/9 [==============================] - 0s 2ms/step - loss: 176.6405 - mean_absolute_error: 176.6405



In [45]:
Total_Round = 1000

for round in range(Total_Round):
  print("<<ROUND ",round+1,">>")
  print("")
  server_state = federated_algorithm.next(server_state, federated_train_data)
  train_evaluate(server_state,Client_num)
  print("")

<<ROUND  1 >>

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

Client4:  
9/9 [==============================] - 0s 3ms/step - loss: 11.3733 - mean_absolute_error: 11.3733


<<ROUND  668 >>

Client1:  
10/10 [==============================] - 0s 2ms/step - loss: 10.8893 - mean_absolute_error: 10.8893

Client2:  
9/9 [==============================] - 0s 2ms/step - loss: 10.5781 - mean_absolute_error: 10.5781

Client3:  
9/9 [==============================] - 0s 3ms/step - loss: 10.0606 - mean_absolute_error: 10.0606

Client4:  
9/9 [==============================] - 0s 2ms/step - loss: 10.4216 - mean_absolute_error: 10.4216


<<ROUND  669 >>

Client1:  
10/10 [==============================] - 0s 2ms/step - loss: 11.0246 - mean_absolute_error: 11.0246

Client2:  
9/9 [==============================] - 0s 2ms/step - loss: 10.8664 - mean_absolute_error: 10.8664

Client3:  
9/9 [==============================] - 0s 3ms/step - loss: 9.4583 - mean_absolute_error: 9.4583

Client4:  
9/9 [=============================

In [46]:
test_evaluate(server_state,Client_num)

Client1:  
10/10 [==============================] - 0s 2ms/step - loss: 10.7333 - mean_absolute_error: 10.7333

Client2:  
9/9 [==============================] - 0s 2ms/step - loss: 10.6811 - mean_absolute_error: 10.6811

Client3:  
9/9 [==============================] - 0s 2ms/step - loss: 9.1023 - mean_absolute_error: 9.1023

Client4:  
9/9 [==============================] - 0s 2ms/step - loss: 10.7604 - mean_absolute_error: 10.7604



In [47]:
local_1_train_X =[]
local_1_train_Y =[]
local_2_train_X =[]
local_2_train_Y =[]
local_3_train_X =[]
local_3_train_Y =[]
local_4_train_X =[]
local_4_train_Y =[]
local_test_X = []
local_test_Y = []

for line in federated_train_data[0]:
  local_1_train_X, local_1_train_Y = line

for line in federated_train_data[1]:
  local_2_train_X, local_2_train_Y = line
  
for line in federated_train_data[2]:
  local_3_train_X, local_3_train_Y = line

for line in federated_train_data[3]:
  local_4_train_X, local_4_train_Y = line

for line in federated_test_data[0]:
  local_test_X, local_test_Y = line

In [48]:
local_1_model = build_local_model()
local_2_model = build_local_model()
local_3_model = build_local_model()
local_4_model = build_local_model()

In [49]:
local_1_model.fit(x=local_1_train_X,y=local_1_train_Y, epochs=1000)
print("LOCAL 1 FINISHED")
local_2_model.fit(x=local_2_train_X,y=local_2_train_Y, epochs=1000)
print("LOCAL 2 FINISHED")
local_3_model.fit(x=local_3_train_X,y=local_3_train_Y, epochs=1000)
print("LOCAL 3 FINISHED")
local_4_model.fit(x=local_4_train_X,y=local_4_train_Y, epochs=1000)
print("LOCAL 4 FINISHED")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4/4 [==============================] - 0s 4ms/step - loss: 15.4175 - mae: 15.4175
Epoch 503/1000
4/4 [==============================] - 0s 4ms/step - loss: 15.8074 - mae: 15.8074
Epoch 504/1000
4/4 [==============================] - 0s 5ms/step - loss: 15.8952 - mae: 15.8952
Epoch 505/1000
4/4 [==============================] - 0s 4ms/step - loss: 15.7440 - mae: 15.7440
Epoch 506/1000
4/4 [==============================] - 0s 6ms/step - loss: 15.4684 - mae: 15.4684
Epoch 507/1000
4/4 [==============================] - 0s 3ms/step - loss: 15.3192 - mae: 15.3192
Epoch 508/1000
4/4 [==============================] - 0s 6ms/step - loss: 15.3257 - mae: 15.3257
Epoch 509/1000
4/4 [==============================] - 0s 3ms/step - loss: 15.3528 - mae: 15.3528
Epoch 510/1000
4/4 [==============================] - 0s 6ms/step - loss: 15.3196 - mae: 15.3196
Epoch 511/1000
4/4 [==============================] - 0s 3ms/step - loss: 15.3274 - mae: 15.3274
Epoch 512

In [50]:
print("Local 1 Test Result")
score1 = local_1_model.evaluate(local_test_X, local_test_Y)
print("Local 2 Test Result")
score2 = local_2_model.evaluate(local_test_X, local_test_Y)
print("Local 3 Test Result")
score3 = local_3_model.evaluate(local_test_X, local_test_Y)
print("Local 4 Test Result")
score4 = local_4_model.evaluate(local_test_X, local_test_Y)

Local 1 Test Result
1/1 [==============================] - 0s 127ms/step - loss: 18.9640 - mae: 18.9640
Local 2 Test Result
1/1 [==============================] - 0s 131ms/step - loss: 17.0220 - mae: 17.0220
Local 3 Test Result
1/1 [==============================] - 0s 125ms/step - loss: 17.2087 - mae: 17.2087
Local 4 Test Result


1/1 [==============================] - 0s 136ms/step - loss: 17.2405 - mae: 17.2405
